In [31]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [32]:
# Surrounding parens means only include those meeting the criteria, ie True
# See https://stackoverflow.com/questions/21415661/logical-operators-for-boolean-indexing-in-pandas
frame = {
    'Total Schools': school_data["School ID"].nunique(),
    'Total Students': student_data["Student ID"].nunique(),
    'Total Budget': school_data["budget"].sum(),
    'Average Math Score': student_data['math_score'].mean(),
    'Average Reading Score': student_data['reading_score'].mean(),
    '% Passing Math': (student_data['math_score'].ge(70)).mean() * 100,
    '% Passing Reading': (student_data['reading_score'].ge(70)).mean() * 100,
    # Need to use element-wise logical-and. That is what the & binary operator performs:
    # https://stackoverflow.com/questions/21415661/logical-operators-for-boolean-indexing-in-pandas
    '% Overall Passing': (student_data['math_score'].ge(70) & student_data['reading_score'].ge(70)).mean() * 100
}

# Since DataFrames display multiple rows and the data is all scalar values,
# you must pass an index to use for the source rows, otherwise you get
# ValueError: If using all scalar values, you must pass an index
districtSummaryAnalysis = pd.DataFrame(frame, index=[0])

# Format numbers
districtSummaryAnalysis['Total Students'] = districtSummaryAnalysis['Total Students'].map('{:,}'.format)
districtSummaryAnalysis['Total Budget'] = districtSummaryAnalysis['Total Budget'].map('${:,.2f}'.format)
districtSummaryAnalysis['% Passing Math'] = districtSummaryAnalysis['% Passing Math'].map('{:.6f}'.format)
districtSummaryAnalysis['% Passing Reading'] = districtSummaryAnalysis['% Passing Reading'].map('{:.6f}'.format)
districtSummaryAnalysis['% Overall Passing'] = districtSummaryAnalysis['% Overall Passing'].map('{:.6f}'.format)

districtSummaryAnalysis


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

### Create an overview DataFrame that summarizes key metrics about each school, including:
* School Name
* School Type
* Total Students
* Total School Budget
* Per Student Budget
* Average Math Score
* Average Reading Score
* % Passing Math
* % Passing Reading
* % Overall Passing (The percentage of students that passed math **and** reading.)


In [33]:
schoolSummary = school_data_complete.groupby(['school_name'])
# Need separate groupby to filter only passing students
passingMath = school_data_complete.where(school_data_complete['math_score'] >= 70).groupby(['school_name'])
passingRead = school_data_complete.where(school_data_complete['reading_score'] >= 70).groupby(['school_name'])
passingBoth = school_data_complete.where((school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)).groupby(['school_name'])

frame = {
    'School Type': schoolSummary['type'].unique().str[0], # unique returns a SeriesGroupBy, str strips square brackets ?!
    'Total Students': schoolSummary['size'].unique().str[0],
    'Total School Budget': schoolSummary['budget'].unique().str[0],
    'Per Student Budget': schoolSummary['budget'].unique().str[0] / schoolSummary['Student ID'].nunique(),
    'Average Math Score': schoolSummary['math_score'].sum() / schoolSummary['Student ID'].nunique(),
    'Average Reading Score': schoolSummary['reading_score'].sum() / schoolSummary['Student ID'].nunique(),
    '% Passing Math': passingMath.count()['Student ID'] / schoolSummary.count()['size'].values * 100,
    '% Passing Reading': passingRead.count()['Student ID'] / schoolSummary.count()['size'].values * 100,
    '% Overall Passing': passingBoth.count()['Student ID'] / schoolSummary.count()['size'].values * 100,
}
schoolSummaryAnalysis = pd.DataFrame(frame)

# Format columns
schoolSummaryAnalysis['Total School Budget'] = schoolSummaryAnalysis['Total School Budget'].map('${:,.2f}'.format)
schoolSummaryAnalysis['Per Student Budget'] = schoolSummaryAnalysis['Per Student Budget'].map('${:,.2f}'.format)
# schoolSummaryAnalysis['Average Math Score'] = schoolSummaryAnalysis['Average Math Score'].map('{:,.6f}'.format)
# schoolSummaryAnalysis['% Passing Math'] = schoolSummaryAnalysis['% Passing Math'].map('{:.2f}%'.format)

schoolSummaryAnalysis




,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

In [34]:
schoolSummaryAnalysis.sort_values(by=['% Overall Passing'], ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [35]:
schoolSummaryAnalysis.sort_values(by=['% Overall Passing'], ascending=True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade
Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [36]:
# Create a pandas series for each grade and group each series by school
frosh = student_data[student_data['grade'] == '9th'].groupby(['school_name'])
soph = student_data[student_data['grade'] == '10th'].groupby(['school_name'])
junior = student_data[student_data['grade'] == '11th'].groupby(['school_name'])
senior = student_data[student_data['grade'] == '12th'].groupby(['school_name'])

frame = {
    '9th': frosh['math_score'].mean(),
    '10th': soph['math_score'].mean(),
    '11th': junior['math_score'].mean(),
    '12th': senior['math_score'].mean(),
}
mathScoresByGrade = pd.DataFrame(frame)
mathScoresByGrade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Scores by Grade 

In [37]:
frame = {
    '9th': frosh['reading_score'].mean(),
    '10th': soph['reading_score'].mean(),
    '11th': junior['reading_score'].mean(),
    '12th': senior['reading_score'].mean(),
}
readingScoresByGrade = pd.DataFrame(frame)
readingScoresByGrade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending
Break down school performances based on average Spending Ranges (Per Student), including:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [38]:
# Create bins and labels for pd.cut
cut_labels = ['<$584', '$585-629', '$630-644', '$645-675']
cut_bins = [0, 584, 629, 644, 675]

# right=True means inclusive of right range (begin-end]
school_data_complete['spending_range'] = pd.cut(school_data_complete["budget"] / school_data_complete["size"], bins=cut_bins, labels=cut_labels, right=True)

spendingRanges = school_data_complete.groupby(['spending_range'])

# Group by spending range for passing math and reading scores
passingMathBySpending = school_data_complete.where(school_data_complete['math_score'] >= 70).groupby(['spending_range'])
passingReadingBySpending = school_data_complete.where(school_data_complete['reading_score'] >= 70).groupby(['spending_range'])
passingBothBySpending = school_data_complete.where((school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)).groupby(['spending_range'])


frame = {
    'Average Math Score': spendingRanges['math_score'].mean(),
    'Average Reading Score': spendingRanges['reading_score'].mean(),
    '% Passing Math': passingMathBySpending.count()['Student ID'] / spendingRanges.count()['size'].values * 100,
    '% Passing Reading': passingReadingBySpending.count()['Student ID'] / spendingRanges.count()['size'].values * 100,
    '% Overall Passing': passingBothBySpending.count()['Student ID'] / spendingRanges.count()['size'].values * 100,
}

spendingRangeAnalysis = pd.DataFrame(frame)
spendingRangeAnalysis['Average Math Score'] = spendingRangeAnalysis['Average Math Score'].map('{:.2f}'.format)
spendingRangeAnalysis['Average Reading Score'] = spendingRangeAnalysis['Average Reading Score'].map('{:.2f}'.format)
spendingRangeAnalysis['% Passing Math'] = spendingRangeAnalysis['% Passing Math'].map('{:.2f}'.format)
spendingRangeAnalysis['% Passing Reading'] = spendingRangeAnalysis['% Passing Reading'].map('{:.2f}'.format)
spendingRangeAnalysis['% Overall Passing'] = spendingRangeAnalysis['% Overall Passing'].map('{:.2f}'.format)

spendingRangeAnalysis

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
spending_range,,,,,
<$584,83.36,83.96,93.70,96.69,90.64
$585-629,79.98,82.31,79.11,88.51,70.94
$630-644,77.82,81.30,70.62,82.60,58.84
$645-675,77.05,81.01,66.23,81.11,53.53


## Scores by School Size

In [39]:
# Create bins and labels for pd.cut
cut_labels = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
cut_bins = [0, 1000, 2000, 5000]

# right=True means inclusive of right range (begin-end]
school_data_complete['size_range'] = pd.cut(school_data_complete["size"], bins=cut_bins, labels=cut_labels, right=True)

sizeRanges = school_data_complete.groupby(['size_range'])

# Group by size range for passing math and reading scores
passingMathBySize = school_data_complete.where(school_data_complete['math_score'] >= 70).groupby(['size_range'])
passingReadingBySize = school_data_complete.where(school_data_complete['reading_score'] >= 70).groupby(['size_range'])
passingBothBySize = school_data_complete.where((school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)).groupby(['size_range'])


frame = {
    'Average Math Score': sizeRanges['math_score'].mean(),
    'Average Reading Score': sizeRanges['reading_score'].mean(),
    '% Passing Math': passingMathBySize.count()['Student ID'] / sizeRanges.count()['size'].values * 100,
    '% Passing Reading': passingReadingBySize.count()['Student ID'] / sizeRanges.count()['size'].values * 100,
    '% Overall Passing': passingBothBySize.count()['Student ID'] / sizeRanges.count()['size'].values * 100,
}
sizeRangeAnalysis = pd.DataFrame(frame)
sizeRangeAnalysis

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
size_range,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,90.624267
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,56.574046


## Scores by School Type

In [40]:
# Group by size range for passing math and reading scores
scoresByType = school_data_complete.groupby(['type'])

passingMathByType = school_data_complete.where(school_data_complete['math_score'] >= 70).groupby(['type'])
passingReadByType = school_data_complete.where(school_data_complete['reading_score'] >= 70).groupby(['type'])
passingBothByType = school_data_complete.where((school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)).groupby(['type'])

frame = {
    'Average Math Score': scoresByType['math_score'].mean(),
    'Average Reading Score': scoresByType['reading_score'].mean(),
    '% Passing Math': passingMathByType.count()['Student ID'] / scoresByType.count()['size'].values * 100,
    '% Passing Reading': passingReadByType.count()['Student ID'] / scoresByType.count()['size'].values * 100,
    '% Overall Passing': passingBothByType.count()['Student ID'] / scoresByType.count()['size'].values * 100,
}
typeRangeAnalysis = pd.DataFrame(frame)
typeRangeAnalysis

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
District,76.987026,80.962485,66.518387,80.905249,53.695878
